# 1. Import the necessary packages

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.datasets import fashion_mnist as fm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import plot_model

# 2. Define a function to load and prepare the dataset

In [26]:
def load_dataset():
    (X_train, y_train), (X_test, y_test) = fm.load_data()

    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    X_train = np.expand_dims(X_train, axis=3)
    X_test = np.expand_dims(X_test, axis=3)

    label_binarizer = LabelBinarizer()
    y_train = label_binarizer.fit_transform(y_train)
    y_test = label_binarizer.fit_transform(y_test)

    (X_train, X_val, y_train, y_val) = train_test_split(X_train, y_train, 
                                                        train_size=0.8)
    train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train)))
    val_ds = (tf.data.Dataset.from_tensor_slices((X_val, y_val)))
    test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test)))

    return train_ds, val_ds, test_ds


# 3. Build a network

In [27]:
def build_network():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(filters=20,
               kernel_size=(5,5),
               padding='same',
               strides=(1,1))(input_layer)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2),
                     strides=(2,2))(x)
    x = Dropout(rate=0.5)(x)

    x = Conv2D(filters = 50,
               kernel_size= (5,5),
               padding='same',
               strides=(1,1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2),
                     strides=(1,1))(x)
    x = Dropout(rate=0.5)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=10)(x)

    output = Softmax()(x)

    model = Model(inputs=input_layer, outputs=output)
    return model
    

# 4. Defien a function that takes a model's history

In [28]:
def plot_model_history(model_history, metric, ylim=None):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)

    plt.title(f'{metric.upper()}')
    if ylim is None:
        plt.ylim([0, 1])
    else:
        plt.ylim(ylim)
    
    plt.savefig(f'{metric}.png')
    plt.close()

# 5. Consume the training and validation

In [29]:
BATCH_SIZE = 256
BUFFER_SIZE = 1024

train_dataset, val_dataset, test_dataset = load_dataset()

train_dataset = (train_dataset
                 .shuffle(buffer_size=BUFFER_SIZE)
                 .batch(BATCH_SIZE)
                 .prefetch(buffer_size=BUFFER_SIZE))
val_dataset = (val_dataset
               .batch(BATCH_SIZE)
               .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = test_dataset.batch(BATCH_SIZE)
